## 질환별 식사습관 web scraping - 서울 아산병원
http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyList.do


In [ ]:
# !pip install beautifulsoup4==4.7.1

In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen 
import pandas as pd

In [2]:
# 협심증, 비만, 고혈압, 뇌졸중, 당뇨병, 천식, 아토피성 피부염, 간염 (급성 바이러스성 간염), 빈혈 (철결핍빈혈), 심근경색증

chronic = ['협심증', '비만', '고혈압', '뇌졸중', '당뇨병', '천식', '아토피', '간염', '빈혈', '심근경색증']
urls = ['http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=117', # 협심증/협심증식 
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=80', # 비만/열량조절
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=29', # 고혈압/고혈압식
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=41', # 뇌졸중/뇌졸중식
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=46', # 당뇨병/당뇨병식
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=104', # 천식/천식식딘
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=77', # 아토피/아토피성 피부염식
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=59', # 간염/바이러스성 간염식
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=63', # 빈혈/빈혈
        'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=75'] # 심근경색증/심근경색식

In [3]:
def bs_web(url):
  # urlopen 함수를 통해 web 변수를 생성
  web = urlopen(url)   # urlopen(url).read().decode('utf-8')

  # BeautifulSoup으로 web 페이지상의 HTML 구조를 파싱
  web_page = BeautifulSoup(web, 'html.parser')

  return web_page

In [4]:
pages = {}

for i in range(len(urls)):
  # get url
  url = urls[i]

  # key == disease
  # value == diet description
  # pages[chronic[i]] = bs_web(url).find('div', {'class': 'contDescription'}).get_text()
  pages[chronic[i]] = bs_web(url).find('div', {'class': 'contDescription'})

In [5]:
def make_dict(disease):
  disease = str(disease).split('<dt class="dep2Title">')
  disease = disease[1:len(disease)-1] # 처음과 마지막 (불필요한 정보) 제거
  d_dict = dict(x.split("</dt>\n<dd><p>") for x in disease) # dict로 만들기
  return d_dict

def clean(d_dict):
  for k,v in d_dict.items():
    d_dict[k] = BeautifulSoup(d_dict[k], 'html.parser').get_text().strip() # 불필요한 html 태그 제거
    # html_tags = ["\xa0", "\n", "\r"]
    d_dict[k] = d_dict[k].replace("\r\n", " ")
    d_dict[k] = d_dict[k].replace("\xa0", "")
  return d_dict

In [6]:
disease_dict = {}
disease_dict['angina_pectoris'] = pages['협심증']
disease_dict['obesity'] = pages['비만']
disease_dict['primary_hypertension'] = pages['고혈압']
disease_dict['cerebrovascular_disease'] = pages['뇌졸중']
disease_dict['diabetes'] = pages['당뇨병']
disease_dict['asthma'] = pages['천식']
disease_dict['atopic_dermatitis'] = pages['아토피']
disease_dict['hepatitis'] = pages['간염']
disease_dict['anemia'] = pages['빈혈']
disease_dict['myocardial_infarction'] = pages['심근경색증']

In [7]:
for k,v in disease_dict.items():
  disease_dict[k] = clean(make_dict(disease_dict[k]))

In [8]:
disease_dict['anemia']

{'권장 식품': '색이 짙은 살코기, 생선, 계란, 우유 및 유제품, 색이 짙은 채소, 과일',
 '그 외 주의사항': '조혈모세포이식 직후에는 멸균식(sterile diet)을 하도록 합니다. 1. 동물성 식품은 반드시 익혀서 섭취합니다. 2. 야채는 익혀서, 과일은 세정 후 껍질을 벗겨서 섭취합니다. 3. 시판 제품 중 살균, 멸균 조리된 제품을 이용합니다. 4. 조리된 식품은 2시간 이내에 냉장보관하며, 24시간내 먹도록 합니다. 5. 조리 전, 식사 전에 손을 깨끗이 씻도록 합니다. 6. 가공식품은 유통기한을 확인하도록 합니다.',
 '식사요법의 실제': '1. 규칙적이고 다양한 식품섭취를 합니다. 2. 스테로이드 사용 시 과다한 염분 섭취는 삼가합니다. 3. 구강내 점막염이 발생한 경우 지나치게 차거나 뜨거운 음식, 향이 강한 식품, 신 음식, 거친 음식의 섭취는 삼가하도록 합니다. 4. 매끼 살코기, 생선, 계란 등의 동물성 단백질을 규칙적으로 섭취합니다. 5. 철분 흡수를 돕는 비타민C의 보충을 위해 과일이나 채소류를 규칙적으로 섭취합니다. 6. 철의 흡수를 방해하는 커피, 홍차, 녹차 등은 식사 전후를 피해서 마시도록 합니다.',
 '식사요법의 필요성': '철 결핍성 빈혈은 가장 흔하게 나타나는 빈혈로, 적혈구를 만드는 주요한 영양소인 철의 부족으로 헤모글로빈 합성이 저하되어 발생합니다. 성인의 하루 철 섭취권장량은 남자 10㎎, 여자 14㎎ 로 권고되고 있습니다. 무엇보다 거의 모든 영양소가 혈액을 만드는데 필요하므로 골고루 균형있게 섭취하는 것이 매우 중요합니다.',
 '주의 식품': '카페인 음료, 녹즙, 엑기스, 날 음식'}

In [9]:
rec_food = {}
rec_food = dict(disease_dict)
for disease in rec_food.keys():
  rec_food[disease] = disease_dict[disease]['권장 식품']

In [10]:
rec_food

{'anemia': '색이 짙은 살코기, 생선, 계란, 우유 및 유제품, 색이 짙은 채소, 과일',
 'angina_pectoris': '신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 등 균형잡힌 식사',
 'asthma': '살코기, 생선, 두부, 신선한 채소 및 과일',
 'atopic_dermatitis': '식품 알레르기와 같이 있는 경우 알레르기 원인 식품을 제외한 모든 음식, 특히 단백질과 아연이 풍부한 육류를 충분히 섭취하는 것이 좋습니다',
 'cerebrovascular_disease': "1. 물, 주스, 우유 등 액상 형태의 대부분의 식품은 '점도증진제(음식의 점도 조절용)'를 섞어서 섭취할 수 있습니다. 2. 삼킴 장애가 심하고 씹지 못 할 때는 밥보다는 죽을 쑤어 블렌더로 곱게 갈아 꿀농도만큼 걸쭉하게 드시길 권장합니다. 반찬(고기, 생선살 나물 등)도 익힌 후 죽을 함께 넣고 블렌더로 곱게 갈아 걸쭉하게 드실 수 있습니다. 3. 우유 대신 요거트를 권장합니다.",
 'diabetes': '채소류, 열량이 적은 식품(곤약, 해조류, 버섯류, 우뭇가사리, 등)',
 'hepatitis': '밥, 떡, 감자, 고구마, 과일, 야채, 단백질 식품( 살코기, 생선, 계란, 두부, 콩 등 )',
 'myocardial_infarction': '신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 등 균형잡힌 식사',
 'obesity': '채소류, 해조류, 곤약, 무가당 차음료(녹차 등), 물(보리차, 생수, 탄산수 등)',
 'primary_hypertension': '신선한 채소를 비롯한 잡곡, 어육류, 생선, 두부, 견과류 그리고 제철 과일과 우유 등 균형잡힌 식사'}

In [11]:
rec_food_df = pd.DataFrame(list(zip(rec_food.keys(), rec_food.values())), columns = ['질환명', '질환별 추천 식품'])
rec_food_df.head(3)

,질환명,질환별 추천 식품
0,angina_pectoris,"신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 ..."
1,obesity,"채소류, 해조류, 곤약, 무가당 차음료(녹차 등), 물(보리차, 생수, 탄산수 등)"
2,primary_hypertension,"신선한 채소를 비롯한 잡곡, 어육류, 생선, 두부, 견과류 그리고 제철 과일과 우유..."


## 조리식품 레시피 - 식품 안전나라 공공데이터 API
https://www.foodsafetykorea.go.kr/api/newDatasetDetail.do

In [1]:
import requests
import pandas as pd
import re
# from konlpy.tag import Twitter

### API 사용해서 레시피 (전체) XML로 가져오기

In [5]:
temp = pd.read_csv("images_df.csv",encoding='utf-8')
temp['썸네일']

0       http://www.foodsafetykorea.go.kr/uploadimg/coo...
1       http://www.foodsafetykorea.go.kr/uploadimg/coo...
2       http://www.foodsafetykorea.go.kr/uploadimg/coo...
3       http://www.foodsafetykorea.go.kr/uploadimg/coo...
4       http://www.foodsafetykorea.go.kr/uploadimg/coo...
                              ...                        
1312    http://www.foodsafetykorea.go.kr/uploadimg/201...
1313    http://www.foodsafetykorea.go.kr/uploadimg/201...
1314    http://www.foodsafetykorea.go.kr/uploadimg/202...
1315    http://www.foodsafetykorea.go.kr/uploadimg/202...
1316    http://www.foodsafetykorea.go.kr/uploadimg/202...
Name: 썸네일, Length: 1317, dtype: object

In [13]:
# http://openapi.foodsafetykorea.go.kr/api/keyId/serviceId/dataType/startIdx/endIdx

keyId = '5579b4b6ce5946b693c5'

base_url = 'http://openapi.foodsafetykorea.go.kr/api/'
serviceId = 'COOKRCP01'
dataType = 'xml'
startIdx = '1'
endIdx = '1000'
startIdx2 = '1001'
endIdx2 = '2000'

request_url = base_url + "/" + keyId + "/" + serviceId + "/" + dataType + "/" + startIdx + "/" + endIdx
print(request_url) # 아래 링크를 클릭해 결과가 잘 나오는지 확인합니다.
response = requests.get(request_url).text
soup = BeautifulSoup(response, 'lxml-xml') # Parser가 바뀌었음에 유의합니다.

request_url2 = base_url + "/" + keyId + "/" + serviceId + "/" + dataType + "/" + startIdx2 + "/" + endIdx2
print(request_url2) # 아래 링크를 클릭해 결과가 잘 나오는지 확인합니다.
response2 = requests.get(request_url2).text
soup2 = BeautifulSoup(response2, 'lxml-xml') # Parser가 바뀌었음에 유의합니다.

http://openapi.foodsafetykorea.go.kr/api//5579b4b6ce5946b693c5/COOKRCP01/xml/1/1000
http://openapi.foodsafetykorea.go.kr/api//5579b4b6ce5946b693c5/COOKRCP01/xml/1001/2000


#### 요리명과 재료 추출

In [14]:
def extract_info(tag_name):
  content = soup.find_all(tag_name) # XML의 <>는 HTML의 Tag와 같이 접근 가능
  content += soup2.find_all(tag_name)

  content = [x.get_text() for x in content] # 텍스트 추출로 앞 뒤의 <> 제거

  return content

recipe_names = extract_info('RCP_NM')
ingredients = extract_info('RCP_PARTS_DTLS')

print(len(recipe_names), len(ingredients))

1318 1318


##### 재료 식품명 전처리

In [15]:
# [], (), 속재료(), 반죽재료(), 개, 적당량, 약간
# ',' 으로 split 


# \n 으로 팔로잉 되는 식재료 부제목 지우기 -- "\n"으로 split 후 ','가 없다면 소제목이라는 패턴을 파악함
def remove_subheading(ingredients):

  ingredients = ingredients.split("\n")
  measure_words = ['g', '공기', '큰술', '작은술', '장', '개', '마리', '알', 'cm', 'ml', '약간', '적당량', '조금']
  filtered = []

  for line in ingredients:
    res = [mw for mw in measure_words if (mw in line)]

    # if there is a comma or a mw in a str, it is NOT a subheading
    if ("," in line) or (res):

      # now remove measurewords and extraneous info from the line
      line = remove_mw(line)
      filtered.append(line) # append to the list
      
    else:
      # print("filtered out: ", line)
      pass

  # flatten the list
  filtered = sum(filtered, [])
      
  return filtered


# ingredients가 string으로 주어졌을때 ',' 으로 나뉘어진 재료들 정리
def remove_mw(ingredients):
  ingredients = ingredients.split(",")

  filtered = []
  measure_words = ['약간', '적당량', '조금']

  for ingredient in ingredients:

    ingredient = ingredient.strip()
    res = [mw for mw in measure_words if (mw in ingredient)]
    ingredient = re.sub("[\(\[].*?[\)\]]", "", ingredient).strip() # remove all text in () and/or []
    last = "".join(ingredient.split(" ")[-1]) # "".join(list) --> converts list to str

    if last[:1].isdigit(): # if the last word starts with a number
      ingredient = ingredient.split(" ")[:-1] # remove measure info

    if res: # use if statement to avoid for loop iterating unnecessarily (efficiency-wise)
      for mw in measure_words:
        if mw in ingredient:
          ingredient = ingredient.replace(mw, "").strip() # remove measure word

    filtered.append("".join(ingredient)) # append to list as a string

  return list(set(filtered)) # return a list after converting from a set to remove duplicates

In [16]:
ingredients_filtered = [] # list of ingredients (filtered)
has_error = [] # keeps track of indices that had error

for i in range(len(ingredients)):
  try:
    filtered = remove_subheading(ingredients[i])
    ingredients_filtered.append(filtered)
  except:
    ingredients_filtered.append("placeholder") # insert placeholder when error occurs so the indices are still in order
    has_error.append(i)

In [17]:
# noticed all lines with error has missing commas
# add missing commas
ingredients[975] = '양파 10g, 할라피뇨 3g, 쇠고기갈은것 10g, 토마토콩카세 10g, 양상추 10g, 올리브오일 1g, 또띠아8인치 30g, 아보카도 6g, 사워크림 3g, 소금적당량, 후춧가루 적당량, 강낭콩 40g, 베이컨 2g, 마늘 1g, 양파 10g, 바질마른것 약간, 배 3g, 오렌지 3g, 레몬주스 2g, 파프리카 가루 1g '
ingredients[1024] = '참치캔 15g, 김치 15g, 양파다진것 15g, 부추 5g, 마늘다진것 2g, 깨소금 1g, 참기름 1g, 밥 60g, 달걀 20g, 우유 5g, 소금 적당량, 식용유 적당량, 고추장 2g, 케첩 4g, 마요네즈 1g, 크림치즈 1g, 물 3g '
ingredients[1025] = '쌀국수 40g, 닭안심 30g, 대파 10g, 김 1g, 통후추 적당량, 슬라이스 햄 5g, 오이 20g, 시금치 10g, 불린 미역, 통마늘 5g, 달걀 10g, 참기름 적당량, 소금 적당량, 닭육수 100g, 식초 5g, 설탕 2g, 간장 적당량, 땅콩버터 2g, 소금 적당량 '
ingredients[1026] = '달걀물 10g, 식용유 1g, 홍고추 3g, 마늘다진것 1g, 닭가슴살 12g, 새우 12g, 양파 12g, 청경채 20g, 스파게티면 40g, 방울토마토 15g, 버터 적당량, 간장 3g, 발사믹식초 1g, 굴소스 2g, 꿀 3g '
ingredients[1054] = '흰살 생선 70g, 죽순 20g, 청피망 15g, 홍피망 15g, 건표고버섯 10g, 전분 5g, 치즈슬라이스 10g, 참기름 적당량, 소금 적당량, 흰후춧가루 적당량, 잣 3g, 물 4g, 설탕 3g, 레몬즙 3g, 간장 2g, 소금적당량 '
ingredients[1055] = '쇠고기살코기 40g, 당근 10g, 오이피클 10g, 양파 15g, 깻잎 10g, 밥 50g, 전분 6g, 식용유 적당량, 깨소금 2g, 참기름 2g, 소금 적당량, 간장 2g, 청주 2g, 참기름 1g, 후춧가루 적당량 '
ingredients[1086] = '새우 15g, 생선육수 130g, 베이컨 10g, 양파 10g, 샐러리 5g, 청피망 8g, 마늘다진것 1g, 토마토콩카세 10g, 밀가루 2g, 버터 3g, 당근 5g, 타바스코 적당량, 케이엔페퍼 적당량, 우스터소스 적당량, 후추 적당량, 소금 적당량, 건타임 0.5g, 건바질 0.5g, 건오레가노 0.5g, 월계수잎 0.2g'
ingredients[1108] = '단호박 80g(8*8cm), 적양파 10g(1/10개), 홍피망 5g(1/9개), 청피망 5g(1/9개), 표고버섯 20g(2장), 칵테일 새우 5마리, 우유 10g(2작은술), 식용유 약간, 참깨 5g(1작은술), 마요네즈 5g(1작은술), 간장 2g(1/3작은술), 겨자가루 약간 '

In [18]:
# replace placeholders
for error in has_error:
  filtered = remove_subheading(ingredients[error])
  ingredients_filtered[error] = filtered

#### 조리법 추출

In [19]:
recipes = soup.find_all('row')
recipes += soup2.find_all('row')

recipe_list = []

for content in recipes:
    direction = ''
    for tag in content.find_all():
        if tag.name.startswith('MANUAL') & ('IMG' not in tag.name) :
            direction += tag.get_text()
    
    recipe_list.append(direction)

len(recipe_list)

1318

In [20]:
ingredients_filtered = [', '.join(ele) for ele in ingredients_filtered] # convert list of lists to list of strings

recipe_df = pd.DataFrame(list(zip(recipe_names, ingredients_filtered, recipe_list)),
               columns =['요리명', '재료', '레시피'])

recipe_df

,요리명,재료,레시피
0,칼륨 듬뿍 고구마죽,"설탕, 고구마, 물, 찹쌀가루, 잣",1. 고구마는 깨끗이 씻어서 껍질을 벗기고 4cm 정도로 잘라준다.a2. 찜기에 고...
1,누룽지 두부 계란죽,"표고버섯, 애호박, 당근, 누룽지, 소금, 순두부, 참기름, 흰 후추, 참깨, 달걀","1. 깨끗이 씻어 손질한 애호박, 당근과 기둥을 뗀 표고버섯을 잘게 다지듯이 썬다...."
2,오색지라시 스시,"밥, 설탕, 소금, 식초, 달걀노른자, 양파, 새우, 홍피망, 표고버섯, 청피망, ...",1. 배합초는 중불에서 설탕이 녹을 때까지 저어가면서 녹인다.2. 뜨거운 밥에 배합...
3,두부 곤약 나물 비빔밥,"흰쌀, 찹쌀, 두부, 실곤약, 현미쌀, 콩나물, 고사리, 애호박, 소금, 당근, 표...",1. 고사리는 상태에 따라 2~5시간 정도 불린 후 30분 정도 삶아 찬물에 헹구어...
4,저염 간장을 이용한 닭개장 비빔밥,"쌀, 검은쌀, 월계수잎, 닭가슴살, 후추, , 고사리, 숙주, 토란대, 소금, 들기...",1. 쌀은 30분 정도 불린 후 물을 1:1로 하여 밥을 짓는다.2. 숙주는 끓는 ...
...,...,...,...
1313,얼큰 콩나물 수제비,"콩나물, 황태머리, 소금, 고추장, 밀가루, 고춧가루",1. 밀가루에 소금을 섞어 반죽한 뒤 숙성시킨다.2. 황태머리를 구운 뒤 찬물에 넣...
1314,"단호박 생선탕수, 키위소스","양파, 식초, 키위소스, 동태살, 당근, 오이, 계란, 단호박, 전분, 목이버섯, ...","1. 따뜻한 물에 부린 목이버섯, 동태살, 당근, 오이, 양파를 먹기 좋은 크기로 ..."
1315,낙지모듬초말이,"설탕, 사과, 간장10g, 고춧가루, 식초, 낙지, 표고버섯, 당근, 홍고추, 물,...","1. 낙지는 살짝 데치고 한입 크기로 썬다.2. 오이와 당근, 표고, 가지는 얇게 ..."
1316,삼각김밥 빵,"양파, 르뱅발효종, 소금, 당근, 물, 쌀가루, 먹물가루, 올리브유, 달걀, 우유,...","1. (속재료) 양파, 마늘, 토마토, 파프리카, 닭가슴살을 다져 올리브유를 두른 ..."


In [21]:
print(recipe_df[recipe_df['요리명'] == ''].index) # 비어있는 재료 인덱스
print(recipe_df[recipe_df['재료'] == ''].index) # 비어있는 재료 인덱스
print(recipe_df[recipe_df['레시피'] == ''].index) # 비어있는 재료 인덱스

Int64Index([], dtype='int64')
Int64Index([659, 660, 1146, 1213], dtype='int64')
Int64Index([828], dtype='int64')


In [22]:
## 레시피 확인후 missing ingredients 추가
# print(recipe_df['레시피'][659])
# print(recipe_df['레시피'][660])
# print(recipe_df['레시피'][1146])
# print(recipe_df['레시피'][1213])

ingredients_filtered[659] = ', '.join(['물','멸치','다시마','호박잎','다슬기살','달걀','부추','대파','다진마늘'])
ingredients_filtered[660] = ', '.join(['양배추', '당근', '양파', '표고버섯', '두부', '다진돼지고기','참기름', '설탕', '소금', '후춧가루', '밀가루','다진마늘', '토마토케첩','간장','매실액'])
ingredients_filtered[1146] = ', '.join(['브로콜리', '실파', '그린빈', '당근', '홍배추', '홍파프리카', '양파', '청고추', '파슬리', '밀가루', '우유', '달걀', '버터', '커리파우더', '닭육수', '식용유'])
ingredients_filtered[1213] = ', '.join(['곤약', '콩', '마늘', '마늘쫑', '파프리카', '저나트륨 조림소스'])

In [23]:
# update dataframe to include these ingredients
recipe_df['재료'] = ingredients_filtered

In [24]:
print(recipe_df.loc[828]) # 레시피 없는 row drop (정확한 대체 레시피를 찾기 힘듬)
recipe_df = recipe_df.drop(828)
recipe_df = recipe_df.reset_index(drop=True) # reset index so 828 isnt missing anymore

요리명                                     비타 오이 물김치
재료     설탕, 식초, 무, 배, 당근, 홍고추, 오이, 파프리카, 고춧가루, 미나리
레시피                                              
Name: 828, dtype: object


In [25]:
recipe_df.info() # note: now total num rows == 1317 instead of 1318

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1317 entries, 0 to 1316
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   요리명     1317 non-null   object
 1   재료      1317 non-null   object
 2   레시피     1317 non-null   object
dtypes: object(3)
memory usage: 31.0+ KB


## 질환별 좋은 음식 데이터

In [ ]:
# urls = ['http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=117', # 협심증/협심증식 
#         'http://www.amc.seoul.kr/asan/healthinfo/mealtherapy/mealTherapyDetail.do?mtId=80', # 비만/열량조절
#         'https://www.insunet.co.kr/health-guide/52069', # 고혈압/고혈압식
#         'https://www.insunet.co.kr/health-guide/51931', # 뇌졸중/뇌졸중식
#         'https://www.insunet.co.kr/health-guide/52113', # 당뇨병/당뇨병식
#         'https://www.insunet.co.kr/health-guide/52047', # 천식/천식식딘
#         'https://www.insunet.co.kr/health-guide/52269', # 아토피/아토피성 피부염식
#         'https://www.insunet.co.kr/health-guide/51925', # 간염/바이러스성 간염식
#         'https://www.insunet.co.kr/health-guide/52007', # 빈혈/빈혈
#         'https://www.insunet.co.kr/health-guide/51904',
#         'https://www.insunet.co.kr/health-guide/51997'] # 심근경색증/심근경색식

In [ ]:
# pages = {}

# for i in range(len(urls)):
#   # get url
#   url = urls[i]

#   # key == disease
#   # value == diet description
#   # pages[chronic[i]] = bs_web(url).find('div', {'class': 'contDescription'}).get_text()
#   pages[chronic[i]] = bs_web(url).find('div', {'class': 'contDescription'})

## 식이요법별 df 만들기

In [26]:
rec_food_df.head()

,질환명,질환별 추천 식품
0,angina_pectoris,"신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 ..."
1,obesity,"채소류, 해조류, 곤약, 무가당 차음료(녹차 등), 물(보리차, 생수, 탄산수 등)"
2,primary_hypertension,"신선한 채소를 비롯한 잡곡, 어육류, 생선, 두부, 견과류 그리고 제철 과일과 우유..."
3,cerebrovascular_disease,"1. 물, 주스, 우유 등 액상 형태의 대부분의 식품은 '점도증진제(음식의 점도 조..."
4,diabetes,"채소류, 열량이 적은 식품(곤약, 해조류, 버섯류, 우뭇가사리, 등)"


In [27]:
recipe_df.head()

,요리명,재료,레시피
0,칼륨 듬뿍 고구마죽,"설탕, 고구마, 물, 찹쌀가루, 잣",1. 고구마는 깨끗이 씻어서 껍질을 벗기고 4cm 정도로 잘라준다.a2. 찜기에 고...
1,누룽지 두부 계란죽,"표고버섯, 애호박, 당근, 누룽지, 소금, 순두부, 참기름, 흰 후추, 참깨, 달걀","1. 깨끗이 씻어 손질한 애호박, 당근과 기둥을 뗀 표고버섯을 잘게 다지듯이 썬다...."
2,오색지라시 스시,"밥, 설탕, 소금, 식초, 달걀노른자, 양파, 새우, 홍피망, 표고버섯, 청피망, ...",1. 배합초는 중불에서 설탕이 녹을 때까지 저어가면서 녹인다.2. 뜨거운 밥에 배합...
3,두부 곤약 나물 비빔밥,"흰쌀, 찹쌀, 두부, 실곤약, 현미쌀, 콩나물, 고사리, 애호박, 소금, 당근, 표...",1. 고사리는 상태에 따라 2~5시간 정도 불린 후 30분 정도 삶아 찬물에 헹구어...
4,저염 간장을 이용한 닭개장 비빔밥,"쌀, 검은쌀, 월계수잎, 닭가슴살, 후추, , 고사리, 숙주, 토란대, 소금, 들기...",1. 쌀은 30분 정도 불린 후 물을 1:1로 하여 밥을 짓는다.2. 숙주는 끓는 ...


In [28]:
rec_food_df = rec_food_df.set_index('질환명')
rec_food_df.head()

,질환별 추천 식품
질환명,
angina_pectoris,"신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 ..."
obesity,"채소류, 해조류, 곤약, 무가당 차음료(녹차 등), 물(보리차, 생수, 탄산수 등)"
primary_hypertension,"신선한 채소를 비롯한 잡곡, 어육류, 생선, 두부, 견과류 그리고 제철 과일과 우유..."
cerebrovascular_disease,"1. 물, 주스, 우유 등 액상 형태의 대부분의 식품은 '점도증진제(음식의 점도 조..."
diabetes,"채소류, 열량이 적은 식품(곤약, 해조류, 버섯류, 우뭇가사리, 등)"


In [29]:
for disease in rec_food_df.index:
  print(rec_food_df.loc[disease]['질환별 추천 식품'],"\n")

신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 등 균형잡힌 식사 

채소류, 해조류, 곤약, 무가당 차음료(녹차 등), 물(보리차, 생수, 탄산수 등) 

신선한 채소를 비롯한 잡곡, 어육류, 생선, 두부, 견과류 그리고 제철 과일과 우유 등 균형잡힌 식사 

1. 물, 주스, 우유 등 액상 형태의 대부분의 식품은 '점도증진제(음식의 점도 조절용)'를 섞어서 섭취할 수 있습니다. 2. 삼킴 장애가 심하고 씹지 못 할 때는 밥보다는 죽을 쑤어 블렌더로 곱게 갈아 꿀농도만큼 걸쭉하게 드시길 권장합니다. 반찬(고기, 생선살 나물 등)도 익힌 후 죽을 함께 넣고 블렌더로 곱게 갈아 걸쭉하게 드실 수 있습니다. 3. 우유 대신 요거트를 권장합니다. 

채소류, 열량이 적은 식품(곤약, 해조류, 버섯류, 우뭇가사리, 등) 

살코기, 생선, 두부, 신선한 채소 및 과일 

식품 알레르기와 같이 있는 경우 알레르기 원인 식품을 제외한 모든 음식, 특히 단백질과 아연이 풍부한 육류를 충분히 섭취하는 것이 좋습니다 

밥, 떡, 감자, 고구마, 과일, 야채, 단백질 식품( 살코기, 생선, 계란, 두부, 콩 등 ) 

색이 짙은 살코기, 생선, 계란, 우유 및 유제품, 색이 짙은 채소, 과일 

신선한 채소를 비롯한 잡곡, 저지방 어육류, 생선, 두부 그리고 제철 과일과 우유 등 균형잡힌 식사 



In [ ]:
# grain = [] # 잡곡
# low_fat_fish_meat = [] # 저지방 어육류
# fruit = [] # (제철) 과일
# veg = [] # 채소(류), 야채
# fish = [] # 생선
# nuts = [] # 견과류
# seaweed = [] # 해조류
# mushroom = [] # 버섯류
# meat = [] # 육류, 고기
# fish_meat = fish.append(meat) # 어육류
# ? = [] # 색이 짙은 살코기
# ? = [] # 색이 짙은 채소


### similarity between recommended diet per disease and ingredients from recipe df
#### cosine similarity (list of ingredients)

In [30]:
from collections import Counter
import math

# list_A = rec_food_df['obesity']['질환별 추천 식품'].split(",")
# list_B = recipe_df[2]['재료']
list_A = rec_food_df['질환별 추천 식품']['angina_pectoris'].split(",")
list_B = recipe_df['재료'][1]

counterA = Counter(list_A)
counterB = Counter(list_B)


def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    cos_sim = dotprod / (magA * magB)
    return cos_sim

print(counter_cosine_similarity(counterA, counterB) * 100)

0.0


In [31]:
cossim = []
for disease in rec_food_df.index:
  disease_sim = []

  list_A = rec_food_df['질환별 추천 식품'][disease].split(",")
  counterA = Counter(list_A)

  for i in range(len(recipe_df)):
    list_B = recipe_df['재료'][i]
    counterB = Counter(list_B)

    disease_sim.append(counter_cosine_similarity(counterA, counterB) * 100)

  cossim.append(disease_sim)

In [32]:
len(disease_sim)

1317

In [33]:
len(cossim)

10

In [34]:
for i in range(len(cossim)):
  if sum(cossim[i]) > 0:
    print(list(rec_food.keys())[i])

hepatitis


## 요리명이 input 되었을 때 비슷한 요리 찾기 (재료 cosine similarity 기준)

#### upper right triangle df 으로 cosine similarity 저장 (col & row name == 요리명)

In [ ]:
# dish = list(recipe_df['요리명'])

# cossim = pd.DataFrame(index = dish, columns = dish)

# for i in range(len(recipe_df) - 1):
#   dishA = dish[i]
#   ingredientsA = recipe_df['재료'][i]
#   counterA = Counter(ingredientsA)

#   for j in range(i+1, len(recipe_df)):
#     dishB = dish[j]
#     ingredientsB = recipe_df['재료'][j]
#     counterB = Counter(ingredientsB)

#     cossim.loc[dishA][dishB] = counter_cosine_similarity(counterA, counterB) * 100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
# cossim.head()

,칼륨 듬뿍 고구마죽,누룽지 두부 계란죽,오색지라시 스시,두부 곤약 나물 비빔밥,저염 간장을 이용한 닭개장 비빔밥,취나물 비빔밥 & 청국장 쌈장,해물 채소장을 넣은 곰취 쌈밥,단호박 케일밥,오색영양밥 롤,새우 두부 계란찜,부추 콩가루 찜,연근 구기자 물김치,방울토마토 소박이,순두부 사과 소스 오이무침,사과 새우 북엇국,사탕절편 떡국,두부 표고 완자탕,저염 된장으로 맛을 낸 황태해장국,된장국,표고버섯 청경채국,치커리샐러드와 올리브 마늘 소스,구운가지 닭가슴살 샐러드와 들깨소스,스트로베리 샐러드,유자꿀드레싱을 곁들인 곤드레 단호박 크로켓 샐러드,시금치 우유 소스와 그린매쉬드포테이토,모과 소스를 곁들인 채소두부,버섯구이와 두부타르타르 소스,새우살 두부완자 샐러드와 매실 생강 소스,구운채소와 간장레몬 소스,브로콜리 컬리플라워 샐러드와 두유 요거트 소스,비빔 메밀냉채,약재 소스를 곁들인 닭가슴살롤,사과 레몬 대구 구이,유자 소스 동태살 구이,비네그레트 드레싱을 이용한 관자요리,곤드레도우와 레몬허니 소스 피자,청국장 실곤약 파스타,고소하고 담백한 너트국수,메밀꽃 필 국수,적양배추 순두부 푸딩,...,만가닥버섯볶음,양파홍초절임,고구마순나물,저염 겉절이,아욱 된장무침,검은콩부추전,고사리파전&단감전,바지락매생이전,알콩달콩전,계란만두&토마토소스,통도라지양념구이,삼색계란찜,고구마순들깨나물,시금치토마토무침,바나나미숫가루쉐이크,배쨈 식빵,사과 견과류 식빵,카카오 깜빠뉴,곶감 베이글,단호박크림을 곁들인 브로콜리 베이글,어묵 잡채,토마토 비빔국수,멸치 전복 저염국수,유부초절임,토마토 미소 된장국,미역 미소국,포두부채소잡채,속이 든든든한 밥 빵,참외지 리코타버무리,과일 피클 젤리,바나나 쑥두부푸딩,채소말이와 고구마가지구이,들깨 곤약 냉채,요거트 단호박 샐러드,홍합 배춧국,얼큰 콩나물 수제비,"단호박 생선탕수, 키위소스",낙지모듬초말이,삼각김밥 빵,병아리콩 컵케이크
칼륨 듬뿍 고구마죽,NaN,81.0302,82.415,85.5558,86.1997,81.1104,88.225,86.3015,86.3403,78.6451,82.2787,83.381,86.6249,67.9639,82.2217,84.1096,85.8276,80.2501,84.7476,81.4417,81.127,80.7296,75.1007,84.157,74.7532,84.0935,77.8237,81.8239,81.1665,80.2189,82.8259,85.0473,80.4753,81.5876,80.6979,77.8204,80.3471,82.8543,87.0622,81.0469,...,75.6578,78.5339,86.5849,87.1602,77.4855,83.8885,87.7846,79.2406,83.7351,82.347,80.8433,76.2652,81.9529,77.4797,62.974,77.64,76.6907,77.9712,77.154,76.4084,81.3332,85.3694,85.2493,83.5389,81.9813,61.8463,81.703,81.1924,78.1788,83.5518,76.715,85.5618,84.6367,79.845,80.8241,82.7624,82.0986,88.0531,88.1031,78.2019
누룽지 두부 계란죽,NaN,NaN,91.3164,93.1477,91.8264,87.4273,92.3736,86.4722,93.1658,85.4098,86.8738,88.3141,86.4145,76.8852,87.8058,87.1804,91.4447,81.7057,86.039,85.16,84.6207,83.2684,82.5315,89.2096,79.3168,89.0782,86.5082,86.9302,86.4383,86.3221,87.3266,90.7385,83.3238,88.6658,85.8627,82.8115,85.7238,87.0784,88.7764,86.3141,...,83.4869,79.9584,89.5547,88.4595,88.0145,90.8618,85.8214,79.6819,84.4147,88.8569,89.7924,83.1068,82.9515,82.0724,60.1585,84.3182,85.5182,84.8365,77.921,81.6692,88.3291,86.6253,93.0229,87.7772,87.8327,66.0466,87.7508,83.8898,83.5614,85.2978,84.4926,89.9239,87.9834,86.9934,87.0447,79.6819,89.2509,91.3252,89.0298,82.666
오색지라시 스시,NaN,NaN,NaN,90.1217,90.4065,86.1755,94.1884,86.566,91.8794,86.998,84.5472,88.917,86.7227,75.0322,90.9545,85.7143,93.031,81.9515,87.8039,86.7528,87.4391,83.7519,82.7422,90.5868,82.8123,91.7844,89.6531,87.7808,90.3805,87.4818,88.8796,91.9973,84.9297,94.355,88.8499,83.8369,86.4248,86.1601,90.4552,86.4934,...,85.7244,83.3517,90.5373,88.6645,84.8458,87.1942,86.1461,79.9,85.6568,89.0132,87.2227,83.5096,78.5714,79.3168,63.7298,86.2422,87.5394,86.4673,79.2527,81.1554,86.5173,86.1084,92.9322,89.5697,88.2448,64.728,92.9172,83.362,86.1607,87.7418,84.3739,90.4992,88.3883,85.7527,86.7315,78.1504,90.1949,91.1029,89.8072,83.8583
두부 곤약 나물 비빔밥,NaN,NaN,NaN,NaN,93.3875,87.9788,93.5912,87.6434,92.7036,83.1,86.9511,89.7212,87.404,75.9809,89.6084,87.494,91.2936,85.4618,87.4564,86.4569,85.1976,85.3814,86.4857,89.6656,80.214,90.4007,86.8089,88.8107,88.2571,88.757,88.434,92.1569,84.136,88.7915,86.8739,85.3936,88.8989,86.7442,89.7309,86.5029,...,84.0776,83.9304,88.9577,87.8267,86.6075,90.8423,89.3337,79.3942,86.7837,88.3583,89.7867,84.3825,80.5342,78.42,62.9026,85.2428,85.2322,86.2701,81.0358,82.719,89.4606,88.7312,92.1669,87.7306,89.7065,69.7976,86.3148,85.7167,83.8898,87.6533,85.6809,88.9427,89.8486,88.5577,88.5324,85.2392,88.8982,92.6832,89.7779,83.3339
저염 간장을 이용한 닭개장 비빔밥,NaN,NaN,NaN,NaN,NaN,87.9691,94.0986,88.8567,92.9733,84.2126,90.1452,90.0951,89.9498,75.0786,88.4657,90.4858,91.9838,85.3803,88.9348,84.9862,86.3605,88.6204,85.6091,90.6602,80.6158,91.3388,87.0955,89.283,88.6127,86.8748,91.0344,94.193,87.4223,88.9952,88.1398,83.4264,89.5566,88.0593,92.0958,87.2395,...,86.4393,84.9939,89.4669,91.0228,87.9205,93.0339,92.8943,83.853,87.5785,92.2388,91.1191,83.7314,83.8242,82.04,63.938,84.858,85.659,85.0154,80.8775,82.7399,90.145,92.5582,93.5413,88.1991,89.6841,67.4981,87.1627,

#### dict 형태로 cosine similarity 저장
({key: {key:value, key:value, key:value, ...}, key: {key:value, key:value, key:value, ...}, ...})

In [35]:
dish = list(recipe_df['요리명'])

# initiate a dict where keys are '요리명' (all values initialized to None)
cos_sim = {key: None for key in dish}
# initiate nested dicts within each key
for key in cos_sim.keys():
  cos_sim[key] = {key: None for key in dish}

for i in range(len(recipe_df)):
  dishA = dish[i]
  ingredientsA = recipe_df['재료'][i]
  counterA = Counter(ingredientsA)

  for j in range(len(recipe_df)):
    dishB = dish[j]
    ingredientsB = recipe_df['재료'][j]
    counterB = Counter(ingredientsB)

    cos_sim[dishA][dishB] = counter_cosine_similarity(counterA, counterB) * 100

In [36]:
# print(cos_sim['칼륨 듬뿍 고구마죽'])

In [37]:
#  list({k: v for k, v in sorted(cos_sim['칼륨 듬뿍 고구마죽'].items(), key=lambda item: item[1])})[-6:-1]
# list({k: v for k, v in sorted(cos_sim['칼륨 듬뿍 고구마죽'].items(), key=lambda item: item[1],reverse=True)})[1:6]

In [38]:
# finds top 5 similar dish (based on what ingredients are used) when given dish_name(str) and cos_sim(dict)
def find_similar_recipe(dish_name, cos_sim):

  similar = list({k: v for k, v in sorted(cos_sim[dish_name].items(), key=lambda item: item[1],reverse=True)})[1:6]

  return similar

In [40]:
dish[:10] # example 요리명 for input below

['칼륨 듬뿍 고구마죽',
 '누룽지 두부 계란죽',
 '오색지라시 스시',
 '두부 곤약 나물 비빔밥',
 '저염 간장을 이용한 닭개장 비빔밥',
 '취나물 비빔밥 & 청국장 쌈장',
 '해물 채소장을 넣은 곰취 쌈밥',
 '단호박 케일밥',
 '오색영양밥 롤',
 '새우 두부 계란찜']

In [39]:
dish_name = input('요리명을 입력하세요: ').strip()

find_similar_recipe(dish_name, cos_sim)

요리명을 입력하세요: 단호박 케일밥


['녹차 약밥', '잡곡강정밥', '약밥 닮은 호떡', '된장크림소스 잡곡 오므라이스', '사태떡찜']